In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../1. Preprocessing /data_preprocessed.csv').drop(columns=['Unnamed: 0'])
data.head()

,0
0,"['ученый', 'который', 'совершать', 'революция'..."
1,"['великий', 'император', 'эфир', 'матч', 'тв',..."
2,"['апрель', 'пройти', 'акция', 'монетный', 'нед..."
3,"['азия', 'никуда', 'полет', 'иркутск', 'владив..."
4,"['ия', 'экран', 'появляться', 'первый', 'подро..."


In [3]:
data['0'].values[0]

"['ученый', 'который', 'совершать', 'революция', 'лечение', 'рассеянный', 'склероз', 'получать', 'научный', 'оскар']"

In [4]:
data = data['0'].apply(lambda x: x.strip('[\']').split('\', \''))
data

0        [ученый, который, совершать, революция, лечени...
1        [великий, император, эфир, матч, тв, интервью,...
2        [апрель, пройти, акция, монетный, неделя, росс...
3        [азия, никуда, полет, иркутск, владивосток, ко...
4        [ия, экран, появляться, первый, подробность, с...
                               ...                        
16217    [согласовывать, отпуск, наш, счет, возвращать,...
16218                                    [рубрика, директ]
16219    [рома, опыт, камингаут, сравнительно, небольшо...
16220    [гранд, должный, обыгрывать, аутсайдер, кубок,...
16221                                    [главное, крипта]
Name: 0, Length: 16222, dtype: object

# 2. Извлечение признаков:

После предварительной обработки наступает время преобразования текстовых данных в числовой формат, подходящий для кластеризации.  
Это можно сделать любым из следующих популярных способов:

- 2.1 **«Мешок слов» (BoW)**: представление текстов в виде группы слов и их частоты
- 2.2 **Tермин «Частота»:** обратная частота документа (TF-IDF) — это показатель, который показывает, насколько значимо данное слово в конкретном документе по сравнению с другими документами
- 2.3 **Использование предварительно обученных моделей:** таких как Word2Vec (Миколов и др., 2013), GloVe и BERT для представления слов в виде плотных векторов, отражающих семантические связи.

## 2.1 Bag of words

https://www.analyticsvidhya.com/blog/2021/08/a-friendly-guide-to-nlp-bag-of-words-with-python-example/

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?",
]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names_out())  # Список слов (фич)
print(X.toarray())  # Матрица "документ × слово" (количество вхождений)

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


In [13]:
# sklearn осуществляет препроцессинг по умолчанию, поэтому придется соединить массивы в строку

data_test = data.copy(deep=True)
data_test = data_test.apply(lambda x: ' '.join(x))
data_test = list(data_test.values)
data_test[0:5]

['ученый который совершать революция лечение рассеянный склероз получать научный оскар',
 'великий император эфир матч тв интервью сам франческо тотти легендарный римлянин приезжать церемония премия рб который пройти завтра заглядывать мы гость вопрос черданцев ответ тотти ловить эфир матч',
 'апрель пройти акция монетный неделя россиянин смочь бесплатно обменять монета банкнота зачислять сумма счет сообщать центробанк обмен доступный тыс отделение банк тыс торговый точка весь страна',
 'азия никуда полет иркутск владивосток корея индонезия малайзия вьетнам филиппины багаж рубль тудаобратно везти никакой виза вылет маеиюнь',
 'ия экран появляться первый подробность смартфон бывший главный дизайнер джонни айва известно девайс антисмартфон получать голосовой управление экран заточить работа ия быть функция смартфон источник пока мочь придумывать сравнивать гаджет ждать анонс']

In [15]:

vectorizer = CountVectorizer(stop_words=None)
X = vectorizer.fit_transform(data_test)

print(vectorizer.get_feature_names_out())  # Список слов (фич)
print(X.toarray())  # Матрица "документ × слово" (количество вхождений)
print(X.toarray().sum(axis=1)) # Считаем общее количество вхождений для каждого объекта
print(list(data.apply(lambda x: len(x)))[0:3], list(data.apply(lambda x: len(x)))[-3:]) # Считаем изначальное количество слов у каждого объекта выборки

['аа' 'ааа' 'аааа' ... 'ящик' 'яя' 'ён']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[10 28 26 ... 73 69  2]
[10, 28, 26] [74, 69, 2]


In [17]:
data_lens = data.apply(lambda x: len(x))
x_lens = X.toarray().sum(axis=1)
print(len(data_lens), len(x_lens))
#Количество объектов не изменилось

16222 16222


In [18]:
unmatch_count = 0
for i, data_len in enumerate(data_lens):
    x_len = x_lens[i]
    if data_len != x_len:
        unmatch_count+=1

print(unmatch_count)

1007


Длина слов и сумма строки матрицы не сходятся у некоторых объектов, по всей видимости список стоп-слов, использованных ранее, отличается от используемого в sklearn. Попроубем проверить.

In [ ]:
features = vectorizer.get_feature_names_out()
x_array = X.toarray()
for i, data_len in enumerate(data_lens):
    x_len = x_lens[i]
    if data_len != x_len:
        x_set = set()
        data_set = set(data.values[i])

        for j, feature in enumerate(features):
            if x_array[i][j] == 1:
                x_set.add(feature)
        print(i)
        print(f'Длина до обработки: {data_len}\nДлина после обработки: {x_len}\nРазница: {data_len-x_len}')
        print(f'Длина до обработки без повторения: {len(data_set)}\nДлина после обработки без повторения: {len(x_set)}\nРазница: {len(data_set-x_set)}')
        print(f'Слова до обработки: {data_set}')
        print(f'Слова после обработки: {x_set}')
        print(f'Удаленные слова: {data_set-x_set}')
        assert len(x_set-data_set) == 0 # Проверка на то, что новые слова не добавились
        print('\n'*2)

Как можно видеть хоть параметр stop_words = None, CountVectorizer все равно почему-то удаляет слова, причем чаще наполненных смыслом

In [23]:
len(features)

37248

## 2.2 Tермин «Частота»

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример документов
documents = [
    "Это первый документ.",
    "Этот документ второй.",
    "И это третий документ.",
    "Первый ли это документ?",
]

# Создание векторизатора TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

# Вывод терминов и матрицы TF-IDF
print("Термины:", vectorizer.get_feature_names_out())
print("Матрица TF-IDF:\n", tfidf_matrix.toarray())

Термины: ['второй' 'документ' 'ли' 'первый' 'третий' 'это' 'этот']
Матрица TF-IDF:
 [[0.         0.4574528  0.         0.69113141 0.         0.55953044
  0.        ]
 [0.66338461 0.34618161 0.         0.         0.         0.
  0.66338461]
 [0.         0.40264194 0.         0.         0.77157901 0.49248889
  0.        ]
 [0.         0.34399327 0.65919112 0.51971385 0.         0.42075315
  0.        ]]


In [33]:
vectorizer.get_feature_names_out()

array(['второй', 'документ', 'ли', 'первый', 'третий', 'это', 'этот'],
      dtype=object)

In [35]:

# Создание векторизатора TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data_test)

# Вывод терминов и матрицы TF-IDF
print("Термины:", vectorizer.get_feature_names_out(), len(vectorizer.get_feature_names_out()))
print("Матрица TF-IDF:\n", tfidf_matrix.toarray())

Термины: ['аа' 'ааа' 'аааа' ... 'ящик' 'яя' 'ён'] 37248
Матрица TF-IDF:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 2.3 Использование предварительно обученных моделей

### 2.3.1 Word2Vec

По документации пробуем библиотеку на простом корпусе https://radimrehurek.com/gensim/models/word2vec.html

In [40]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Пример текстового корпуса (список предложений)
sentences = [
    "Word2Vec преобразует слова в векторы.",
    "Векторы сохраняют семантические связи между словами.",
    "Word2Vec используется в NLP для word embedding.",
    "Примеры включают машинное обучение и обработку естественного языка."
]

# Токенизация предложений
tokenized_sentences = [simple_preprocess(sentence, deacc=True) for sentence in sentences]

# Обучение модели Word2Vec
model = Word2Vec(
    sentences=tokenized_sentences,
    vector_size=100,     # Размерность вектора
    window=5,            # Размер окна контекста
    min_count=1,          # Минимальная частота слова для учета
    workers=4,            # Количество потоков
    epochs=50            # Количество эпох обучения
)

# Получение вектора для слова
word_vector = model.wv["преобразует"]
print(f"Вектор слова 'слова': {word_vector[:5]}...")  # Вывод первых 5 значений

# Поиск похожих слов
similar_words = model.wv.most_similar("преобразует", topn=3)
print("Похожие слова на 'преобразует':", similar_words)

Вектор слова 'слова': [-0.00713402  0.00124788 -0.00718346 -0.00223246  0.00373751]...
Похожие слова на 'преобразует': [('для', 0.25297728180885315), ('векторы', 0.17048844695091248), ('nlp', 0.1501290202140808)]


Обучаем модель на нашем датасете

In [43]:
data_test = data.copy(deep=True)

# Обучение модели Word2Vec
model = Word2Vec(
    sentences=data_test,
    vector_size=100,     # Размерность вектора
    window=5,            # Размер окна контекста
    min_count=1,          # Минимальная частота слова для учета
    workers=4,            # Количество потоков
    epochs=50            # Количество эпох обучения
)


Проверяем работу модели смотря близки ли по смыслу близкие в полученном пространестве вектора

In [45]:
# Получение вектора для слова
word_vector = model.wv["чай"]
print(f"Вектор слова 'чай': {word_vector[:5]}...")  # Вывод первых 5 значений

# Поиск похожих слов
similar_words = model.wv.most_similar("чай", topn=3)
print("Похожие слова на 'чай':", similar_words)

Вектор слова 'чай': [-0.8308413   1.0660404   2.5957007  -0.7834279  -0.08392825]...
Похожие слова на 'чай': [('овсянка', 0.6867110729217529), ('трава', 0.6736602783203125), ('ветчина', 0.6692957282066345)]


Здесь я хотел попробовать что-то похожее на знаменитый пример про короля и королеву, только с "кофе" и "молоко". Я сложил вектора этих слов и ожидал получить "латте", однако такого не вышло и кажется эти слова и так слишком близко друг к другу. Я пробовал на других словах, но хорошего результата не поучилось. Предполагаю, что это из-за того, что датасет довольно маленький.

In [47]:
# Получение вектора для слова
word_vector1 = model.wv["молоко"]
print(f"Вектор слова 'молоко': {word_vector[:5]}...")  # Вывод первых 5 значений

word_vector2 = model.wv["кофе"]
print(f"Вектор слова 'кофе': {word_vector[:5]}...")  # Вывод первых 5 значений


# Поиск похожих слов
similar_words = model.wv.most_similar(word_vector1+word_vector2, topn=3)
print("Похожие слова на полученный вектор:", similar_words)

Вектор слова 'молоко': [-0.8308413   1.0660404   2.5957007  -0.7834279  -0.08392825]...
Вектор слова 'кофе': [-0.8308413   1.0660404   2.5957007  -0.7834279  -0.08392825]...
Похожие слова на полученный вектор: [('кофе', 0.913850724697113), ('молоко', 0.8812078237533569), ('американо', 0.75205397605896)]


Усреднение векторов слов для каждого документа в корпусе

In [49]:
def text_to_vector(model, words):
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

Проверяем как сработало усреднение на близких словах к полученному для каждого документа вектору

In [51]:
data_vectors_word2vec = [text_to_vector(model, words) for words in data_test]

In [52]:
print(data_test[2])
model.wv.most_similar(data_vectors_word2vec[2], topn=5)

['апрель', 'пройти', 'акция', 'монетный', 'неделя', 'россиянин', 'смочь', 'бесплатно', 'обменять', 'монета', 'банкнота', 'зачислять', 'сумма', 'счет', 'сообщать', 'центробанк', 'обмен', 'доступный', 'тыс', 'отделение', 'банк', 'тыс', 'торговый', 'точка', 'весь', 'страна']


[('акция', 0.6212311387062073),
 ('платеж', 0.5912837386131287),
 ('банк', 0.5807708501815796),
 ('тыс', 0.5740810036659241),
 ('торги', 0.5729284882545471)]

In [59]:
print(data_test[5])
model.wv.most_similar(data_vectors_word2vec[5], topn=5)

['искать', 'молодежный', 'тариф', 'игра', 'музыка', 'рестик', 'мтс', 'прогрессор', 'помощь', 'это', 'единый', 'хаб', 'развлечение', 'отдых', 'скачивать', 'приложение', 'рубиться', 'игра', 'получать', 'электронный', 'валюта', 'прокоин', 'смочь', 'поменять', 'скидка', 'промокод', 'пакет', 'интернет', 'пополнять', 'другой', 'сервис', 'проголодаться', 'личный', 'подборка', 'редакция', 'ия', 'подсказывать', 'вкусный', 'рестик', 'неподалеку', 'весь', 'это', 'также', 'гб', 'интернет', 'безлимит', 'телега', 'минута', 'смс', 'остаток', 'переноситься', 'подписка', 'мтс', 'музыка', 'конец', 'год', 'рубль', 'месяц', 'пока', 'москва', 'область', 'скоро', 'весь', 'страна', 'подробность']


[('яндекс', 0.6557922959327698),
 ('бесплатно', 0.6203290820121765),
 ('сервис', 0.6081401109695435),
 ('подписка', 0.5977248549461365),
 ('бесплатный', 0.5748127698898315)]

### 2.3.2 Doc2Vec

Перед использованием была изучена эта [статья].(https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e).

Про отличие работы Word2Vec и Doc2Vec:

The model above is called Distributed Memory version of Paragraph Vector (PV-DM). It acts as a memory that remembers what is missing from the current context — or as the topic of the paragraph. While the word vectors represent the concept of a word, the document vector intends to represent the concept of a document.

Here, this algorithm is actually faster (as opposed to word2vec) and consumes less memory, since there is no need to save the word vectors.



 Пример использования из https://radimrehurek.com/gensim/models/doc2vec.html :

In [125]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize

# Пример данных
documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

# Подготовка данных (токенизация и тегирование)
tagged_data = [
    TaggedDocument(words=word_tokenize(doc.lower()), tags=[str(i)]) 
    for i, doc in enumerate(documents)
]

# Обучение модели
model = Doc2Vec(
    vector_size=50,  # Размер вектора
    min_count=2,     # Минимальная частота слова
    epochs=40        # Количество эпох
)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Получение вектора документа
vector = model.infer_vector(word_tokenize("This is a new document."))
print(vector)

[ 0.00079551 -0.00393487  0.00413172 -0.00115991 -0.0082423  -0.00179026
 -0.00620833  0.00801841 -0.00428335 -0.00078557  0.00610221 -0.00211225
  0.00655132  0.0066864  -0.00832772 -0.01053418 -0.00810287 -0.00533939
 -0.01030794 -0.00582843  0.00685461 -0.00220569 -0.00941168  0.00179322
  0.00120937  0.00711207 -0.00242693  0.00938532  0.0083763  -0.00445437
 -0.00070403 -0.0039145  -0.00481523  0.00379924 -0.00446129 -0.000504
 -0.0035168   0.00810844  0.00283994  0.00127982  0.00300739  0.0077778
 -0.00464935  0.00853151 -0.0032797   0.00253399 -0.0073779  -0.00228772
  0.00764174  0.00193785]


Наш датасет:

Размер вектора для документа - **100**

In [142]:
# Подготовка данных (токенизация и тегирование)
tagged_data = [
    TaggedDocument(words=words, tags=[str(i)]) 
    for i, words in enumerate(data_test)
]

# Обучение модели
model = Doc2Vec(
    vector_size=100,  # Размер вектора
    min_count=2,     # Минимальная частота слова
    epochs=40        # Количество эпох
)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Получение вектора документа
vector = model.infer_vector(word_tokenize("Это еще один документ"))
print(vector)

[-5.33696972e-02  3.45010370e-01 -2.41773099e-01 -7.69845545e-02
 -6.87041804e-02 -4.70700711e-01 -1.27060354e-01  3.67884487e-01
 -1.66482568e-01 -1.90216929e-01  3.35039757e-03 -5.47317863e-01
  7.69934952e-02 -7.12446589e-03  1.97469756e-01 -2.57678628e-01
  2.80142695e-01 -4.12734777e-01  2.25550890e-01 -1.79077849e-01
 -6.12685233e-02 -9.92543921e-02  2.09582001e-01  1.12087145e-01
  6.91033244e-01  1.32431731e-01 -2.17868686e-01  3.07495985e-02
 -4.26150769e-01 -7.83714950e-02  5.96661642e-02 -1.94681644e-01
  1.70603376e-02 -2.26324797e-01 -3.89791012e-01  5.80865443e-01
  1.49893165e-01 -6.50546774e-02 -2.63585478e-01 -9.76955965e-02
 -4.38415527e-01 -9.58024785e-02  1.56487986e-01 -2.13762477e-01
 -1.56079948e-01 -2.80722044e-02 -9.43724066e-02 -1.52158663e-01
 -3.20958148e-04  3.46911907e-01 -1.29255921e-01 -4.76593003e-02
  1.84030116e-01  1.39139742e-01  2.53484156e-02 -1.10862516e-01
 -7.12531060e-02 -2.19537511e-01  9.06086192e-02 -1.75159305e-01
 -4.51400243e-02  1.77634

In [143]:
similar_docs = model.dv.most_similar('0', topn=3)  # Ищем документы, похожие на документ с тегом '0'

similar_indexes = [int(tag) for tag, _ in similar_docs]
print(f'Текст "0" {data_test[0]}')
print()
print('Похожие на текст "0" по смыслу тексты:')
for similar in similar_indexes:
    print(f'Текст "{similar}"\n{data_test[similar]}')
    

Текст "0" ['ученый', 'который', 'совершать', 'революция', 'лечение', 'рассеянный', 'склероз', 'получать', 'научный', 'оскар']

Похожие на текст "0" по смыслу тексты:
Текст "15556"
['астроном', 'впервые', 'получать', 'портрет', 'звезда', 'галактика']
Текст "7152"
['вестись', 'эксперимент', 'внедрять', 'ген', 'мышь', 'получать', 'волосатый', 'мамонтовый', 'мышь']
Текст "1241"
['большой', 'розыгрыш', 'продолжаться', 'выполнять', 'условие', 'розыгрыш', 'получать', 'возможность', 'выигрывать', 'квартира', 'машина', 'тысяча', 'другой', 'приз', 'выбирать', 'любимый', 'направление', 'рейс', 'россия', 'санктпетербург', 'москва', 'москва', 'уфа', 'казань', 'москва', 'москва', 'ульяновск', 'сочи', 'самара', 'екатеринбург', 'сочи', 'международный', 'рейс', 'москва', 'гюмри', 'волгоград', 'стамбул', 'владикавказ', 'стамбул', 'москва', 'абудаби', 'махачкала', 'стамбул', 'москва', 'ош', 'цена', 'актуальный', 'момент', 'публикация', 'успевать', 'приобретать', 'приложение']


In [144]:
similar_docs = model.dv.most_similar('5', topn=3)

similar_indexes = [int(tag) for tag, _ in similar_docs]
print(f'Текст "5" {data_test[1]}')
print()
print('Похожие на текст "5" по смыслу тексты:')
for similar in similar_indexes:
    print(f'Текст "{similar}"\n{data_test[similar]}')
    

Текст "5" ['великий', 'император', 'эфир', 'матч', 'тв', 'интервью', 'сам', 'франческо', 'тотти', 'легендарный', 'римлянин', 'приезжать', 'церемония', 'премия', 'рб', 'который', 'пройти', 'завтра', 'заглядывать', 'мы', 'гость', 'вопрос', 'черданцев', 'ответ', 'тотти', 'ловить', 'эфир', 'матч']

Похожие на текст "5" по смыслу тексты:
Текст "11557"
['разыгрывать', 'рубль', 'ответ', 'вопрос', 'мой', 'вещь', 'сделать', 'игра', 'ребенок', 'мой', 'вещь', 'ребята', 'искать', 'предмет', 'картинка', 'забирать', 'подарок', 'предмет', 'спрятать', 'сюрприз', 'рубль', 'деньги', 'беспроводный', 'наушник', 'колонка', 'игровой', 'мышка', 'портативный', 'зарядка', 'еще', 'подсказка', 'пользоваться', 'детский', 'карта', 'копить', 'тратить', 'карманный', 'деньги', 'искать', 'предмет', 'сюрприз', 'детский', 'приложение', 'альфаонлайн', 'март', 'ребенок', 'пока', 'детский', 'карта', 'открывать', 'она', 'смочь', 'каждый', 'день', 'играть', 'вместе']
Текст "12287"
['сбертех', 'запускать', 'облачный', 'среда'

Размер вектора для документа - **50**

In [148]:
# Подготовка данных (токенизация и тегирование)
tagged_data = [
    TaggedDocument(words=words, tags=[str(i)]) 
    for i, words in enumerate(data_test)
]

# Обучение модели
model = Doc2Vec(
    vector_size=50,  # Размер вектора
    min_count=2,     # Минимальная частота слова
    epochs=40        # Количество эпох
)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Получение вектора документа
vector = model.infer_vector(word_tokenize("Это еще один документ"))
print(vector)

[-0.1122893  -0.08361123  0.12109993 -0.0019567  -0.15847522  0.19078478
 -0.03695316  0.66916144 -0.27370995 -0.09069169  0.02462783  0.017155
  0.01840624 -0.28219324  0.1917226   0.31793714  0.2182881  -0.2800033
 -0.11948295 -0.3417047  -0.00791297  0.53922445  0.03335609 -0.08313597
 -0.36942282  0.0285593   0.16309723  0.07219774  0.21331719 -0.07331865
 -0.20728905 -0.44554752 -0.29132205 -0.12919763 -0.5092853   0.02523815
 -0.20494033 -0.30540735  0.44927287 -0.22944018  0.2563658  -0.00779583
 -0.32665238  0.22720411  1.0809518   0.31136936 -0.15323918 -0.01887738
  0.7372884   0.3220741 ]


In [149]:
similar_docs = model.dv.most_similar('0', topn=3)  # Ищем документы, похожие на документ с тегом '0'

similar_indexes = [int(tag) for tag, _ in similar_docs]
print(f'Текст "0" {data_test[0]}')
print()
print('Похожие на текст "0" по смыслу тексты:')
for similar in similar_indexes:
    print(f'Текст "{similar}"\n{data_test[similar]}')
    

Текст "0" ['ученый', 'который', 'совершать', 'революция', 'лечение', 'рассеянный', 'склероз', 'получать', 'научный', 'оскар']

Похожие на текст "0" по смыслу тексты:
Текст "7152"
['вестись', 'эксперимент', 'внедрять', 'ген', 'мышь', 'получать', 'волосатый', 'мамонтовый', 'мышь']
Текст "6278"
['клонирование', 'метод', 'получение', 'несколько', 'идентичный', 'организм', 'путем', 'бесполый', 'число', 'вегетативный', 'размножение', 'клонирование', 'зависимость', 'применение', 'группа', 'живой', 'неживой', 'существо', 'молекулярный', 'репродуктивный', 'терапевтический', 'клонирование', 'мочь', 'подвергаться', 'животное', 'человек', 'группа', 'генетически', 'идентичный', 'организм', 'клетка', 'называться', 'клон', 'термин', 'клонирование', 'приходить', 'русский', 'язык', 'английский', 'первоначально', 'слово', 'клон', 'дргреча', 'веточка', 'побег', 'отпрыск', 'становиться', 'употреблять', 'группа', 'растение', 'получать', 'один', 'растенияпроизводитель', 'вегетативный', 'способ', 'растенияпо

In [150]:
similar_docs = model.dv.most_similar('5', topn=3)

similar_indexes = [int(tag) for tag, _ in similar_docs]
print(f'Текст "5" {data_test[5]}')
print()
print('Похожие на текст "5" по смыслу тексты:')
for similar in similar_indexes:
    print(f'Текст "{similar}"\n{data_test[similar]}')
    

Текст "5" ['искать', 'молодежный', 'тариф', 'игра', 'музыка', 'рестик', 'мтс', 'прогрессор', 'помощь', 'это', 'единый', 'хаб', 'развлечение', 'отдых', 'скачивать', 'приложение', 'рубиться', 'игра', 'получать', 'электронный', 'валюта', 'прокоин', 'смочь', 'поменять', 'скидка', 'промокод', 'пакет', 'интернет', 'пополнять', 'другой', 'сервис', 'проголодаться', 'личный', 'подборка', 'редакция', 'ия', 'подсказывать', 'вкусный', 'рестик', 'неподалеку', 'весь', 'это', 'также', 'гб', 'интернет', 'безлимит', 'телега', 'минута', 'смс', 'остаток', 'переноситься', 'подписка', 'мтс', 'музыка', 'конец', 'год', 'рубль', 'месяц', 'пока', 'москва', 'область', 'скоро', 'весь', 'страна', 'подробность']

Похожие на текст "5" по смыслу тексты:
Текст "10099"
['премиальный', 'карта', 'газпромбанк', 'сочетать', 'комфорт', 'привилегия', 'выгода', 'год', 'газпромбанк', 'премиум', 'получать', 'привилегия', 'спорт', 'путешествие', 'выбор', 'менять', 'каждый', 'месяц', 'кешбэк', 'рубль', 'миля', 'повышенный', 'дох

### 2.3.3 GloVe

In [6]:
from gensim.models import KeyedVectors
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')

# Загрузка русской модели (пример пути)
model_path = "ru_glove_100d.txt"
glove_model = KeyedVectors.load_word2vec_format(model_path, binary=False)

# Корпус на русском
documents = [
    "Привет, как твои дела?",
    "Машинное обучение — это интересно.",
    "Глубокое обучение использует нейронные сети."
]

# Токенизация + лемматизация (опционально)
tokenized_docs = [word_tokenize(doc.lower(), language="russian") for doc in documents]

# Функция для усреднения векторов
def doc_to_vector(tokens, model):
    vectors = []
    for word in tokens:
        if word in model:
            vectors.append(model[word])
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Векторизация
doc_vectors = np.array([doc_to_vector(doc, glove_model) for doc in tokenized_docs])
print(f"Векторы документов: {doc_vectors.shape}")  # (3, 100)

ImportError: cannot import name 'triu' from 'scipy.linalg' (C:\Users\s.yanenkov\AppData\Local\anaconda3\Lib\site-packages\scipy\linalg\__init__.py)

In [8]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [12]:
from gensim.models import KeyedVectors


RuntimeError: Compiled extensions are unavailable. If you've installed from a package, ask the package maintainer to include compiled extensions. If you're building Gensim from source yourself, install Cython and a C compiler, and then run `python setup.py build_ext --inplace` to retry. 

### 2.3.4 BERT